In [1]:
#use LangSmith
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "langhain-ollama-prompts"

In [3]:
prompt = """
Answer the user's query based on the context below.                 
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])



In [5]:
#the same as previous but using MessagePromptTemplates
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

In [6]:
prompt_template.input_variables

['context', 'query']

In [7]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.                 \nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

Invoke LLM with templates

In [8]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2"

#ititialize more determenistic model
llm = ChatOllama(model=model_name, temperature=0.0)

In [9]:
pipeline = prompt_template | llm

In [10]:
context = """LangChain is a framework for developing applications powered by language models. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more."""
query = "What is LangChain used for?"

In [11]:
pipeline.invoke({"context": context, "query": query})

AIMessage(content='LangChain is a framework that can be used for various natural language processing (NLP) tasks, including:\n\n1. Chatbots\n2. Generative Question-Answering (GQA)\n3. Summarization\n\nThese are just a few examples of the many applications LangChain can be used for.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-12-04T09:18:58.91637Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2705030417, 'load_duration': 1591120500, 'prompt_eval_count': 103, 'prompt_eval_duration': 154594583, 'eval_count': 63, 'eval_duration': 660613037, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--581bc0ff-67b3-4ffc-bd53-e2e1b65fda11-0', usage_metadata={'input_tokens': 103, 'output_tokens': 63, 'total_tokens': 166})

Few Shot Prompting

In [16]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

In [13]:
examples = [
    {
        "input": "What is the capital of France?",
        "output": "The capital of France is Paris."
    },
    {
        "input": "Who wrote 'To Kill a Mockingbird'?",
        "output": "'To Kill a Mockingbird' was written by Harper Lee."
    }
]

In [15]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

print(few_shot_prompt.format())

System: What is the capital of France?
AI: The capital of France is Paris.
System: Who wrote 'To Kill a Mockingbird'?
AI: 'To Kill a Mockingbird' was written by Harper Lee.
